# HW3: Variational Autoencoders

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.distributions import Normal
from itertools import chain
from torchlib.generative_model.autoencoder.vae import VAE
from torchlib.dataset.utils import create_data_loader

Enable cuda False


In [2]:
%qtconsole

## VAEs in 2D

### Part A

In [3]:
import numpy as np
def sample_data_1():
    count = 100000
    rand = np.random.RandomState(0)
    return [[1.0, 2.0]] + rand.randn(count, 2) * [[5.0, 1.0]]

def sample_data_2():
    count = 100000
    rand = np.random.RandomState(0)
    return [[1.0, 2.0]] + (rand.randn(count, 2) * [[5.0, 1.0]]).dot(
        [[np.sqrt(2) / 2, np.sqrt(2) / 2], [-np.sqrt(2) / 2, np.sqrt(2) / 2]])

In [4]:
data_1 = sample_data_1().astype(np.float32)
data_1_loader = create_data_loader((data_1,))
data_2 = sample_data_2().astype(np.float32)
data_2_loader = create_data_loader((data_2,))

In [5]:
# visualize data distribution

### Define prior

In [6]:
prior = Normal(loc=torch.zeros(2), scale=torch.ones(2))

### Define encoder

In [7]:
class Encoder(nn.Module):
    def __init__(self, code_size=2):
        super(Encoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(2, 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(8, 8),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.mu = nn.Linear(8, code_size)
        self.logvar = nn.Linear(8, code_size)

    def forward(self, x):
        x = self.model(x)
        mu = self.mu(x)
        logvar = self.logvar(x)
        return Normal(mu, torch.exp(logvar))

### Define decoder

In [8]:
class Decoder_1(nn.Module):
    def __init__(self, code_size=2):
        super(Decoder_1, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(code_size, 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(8, 8),
            nn.LeakyReLU(0.2, inplace=True),
        )
        
        self.mu = nn.Linear(8, code_size)
        self.logvar = nn.Linear(8, code_size)

    def forward(self, x):
        x = self.model(x)
        mu = self.mu(x)
        logvar = self.logvar(x)
        return Normal(mu, torch.exp(logvar))
        
        
class Decoder_2(nn.Module):
    def __init__(self, code_size=2):
        super(Decoder_2, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(code_size, 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(8, 8),
            nn.LeakyReLU(0.2, inplace=True),
        )
        
        self.mu = nn.Linear(8, code_size)
        self.logvar = nn.Linear(8, 1)

    def forward(self, x):
        x = self.model(x)
        mu = self.mu(x)
        logvar = self.logvar(x)
        return Normal(mu, torch.exp(logvar))

### Fit on dataset 1

In [11]:
encoder_type_1_data_1 = Encoder()
decoder_type_1_data_1 = Decoder_1()
optimizer = optim.Adam(chain(encoder_type_1_data_1.parameters(), decoder_type_1_data_1.parameters()), lr=1e-1)
vae_type_1_data_1 = VAE(encoder_type_1_data_1, decoder_type_1_data_1, prior, optimizer)

In [12]:
vae_type_1_data_1.train(num_epoch=10, train_data_loader=data_1_loader)

Epoch 1/10



Totol loss 4.5207/3.2610 (bits/dim) - Negative log likelihood 4.4932/3.2412 (bits/dim) - KL divergence 0.0274/0.0198 (bits/dim)
Epoch 2/10



Totol loss 4.4617/3.2185 (bits/dim) - Negative log likelihood 4.4610/3.2180 (bits/dim) - KL divergence 0.0007/0.0005 (bits/dim)
Epoch 3/10


KeyboardInterrupt: 